In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import pi
from scipy import special as sp
import scipy.integrate as integrate
from scipy.interpolate import CubicSpline, interp2d
from tqdm import tqdm
from scipy.constants import fine_structure, speed_of_light, electron_mass, elementary_charge, hbar, epsilon_0
from scipy import optimize

In [ ]:
#variáveis e constantes
electron_radius = elementary_charge**2/(4*pi*epsilon_0*electron_mass*speed_of_light**2)
num_pon = 100                                                    #número de pontos
mean_ene = 1800                                                    #energia média inicial
std_div = 90                                                       #desvio padrão
mag_field = 2.5*10**6 #[T] intensidade do campo magnético
chi_0 = 1                                                          #valor de chi máximo
tau_e = hbar/(speed_of_light**2*electron_mass)                     #[s] time it takes for light to travel across the electron radius electron_radius/speed_of_light#
n_bins = 5                                                       #número de bins do histograma
t_end = 3                                   #tempos finais
t_steps = 100                                                      #numero de steps

#valores de gamma
gamma_glob = mean_ene + std_div*rng.standard_normal(num_pon)

In [ ]:
def cnst_prop_2(mag):          #conversão entre gamma do eletrão e chi do eletrão
    return mag/(4.5*10**(9))

def bessel_integral(v):
    return integrate.quad(lambda x: sp.kv(5/3, x), v, np.Inf)[0]

#equacao 3.83 de 'Classical and Quantum Description of Plasma and Radiation in Strong Fields' de Fabien Niel
def G_tilde_int(chi, epsilon):
    v = 2 * epsilon / (3 * chi * (1 - epsilon))
    return np.sqrt(3) / (2 * pi) * epsilon * (
                integrate.quad(lambda x: sp.kv(5 / 3, x), v, np.Inf)[
                    0] + 1.5 * epsilon * chi * v * sp.kv(2 / 3, v))

G_tilde_int_vec_aux = lambda chi1, epsilon1: G_tilde_int(chi1, epsilon1)

G_tilde_int_vec = np.vectorize(G_tilde_int_vec_aux)

epsilon_uni = np.linspace(0.0001, 0.9999, 100)

chi_uniform = np.linspace(0.00001, 10, 100)

G_tilde_inter = interp2d(chi_uniform, epsilon_uni, G_tilde_int_vec(chi_uniform, epsilon_uni), kind='cubic')

#equacao 3.88 de 'Classical and Quantum Description of Plasma and Radiation in Strong Fields' de Fabien Niel
def dN_dt(gamma):
    omega_c = elementary_charge * 2.5*10**6 / (electron_mass * mean_ene)
    return (2 / 3) * fine_structure / tau_e / omega_c * integrate.quad(
        lambda x: G_tilde_inter((gamma * cnst_prop_2(2.5*10**6)),
                                  (x / gamma))/(x*gamma), 0, np.Inf)[0]

In [ ]:
def integration(x1):
    return integrate.quad(lambda x: sp.kv(1 / 3, x), x1, np.Inf)[0]

chi_int = np.linspace(0, 1000, int(num_pon))

R = np.array([integration(x) for x in chi_int])

bessel_1_3_int = CubicSpline(chi_int, R)

#equacao 2.3 de 'Classical and Quantum simulation of Extreme Plasma Physics' de Oscar Amaro
def d2P_dtdX(epsilon, gamma):  # funcao analitica nao normalizada de compton scattering
    X_e = cnst_prop_2(2.5*10**6) * gamma
    X_tilde = 2 * epsilon / (3 * X_e * (1 - epsilon))
    return 1/X_e * ((1 - epsilon + 1 / (1 - epsilon)) * sp.kv(2 / 3, X_tilde) - bessel_1_3_int(X_tilde))

def dP_dt(gamma, point):  # CDF
    return integrate.quad(lambda x: d2P_dtdX(x, gamma), 0, point)[0]

def dP_dt_inverse_aux(y, x, gamma):
    return dP_dt(gamma, y) - x

def dP_dt_inverse(x, gamma):
    return optimize.bisect(dP_dt_inverse_aux, 0, 1, args=(x, gamma))

x_uniform = np.linspace(0.000001, 0.9999, 10)

gamma_uni = np.linspace(1, 1900, 10)

chi_uni_2 = np.linspace(0.0001, 1, 10)

dP_dt_inverse_vec_aux = lambda x1, gamma1: dP_dt_inverse(x1, gamma1)

dP_dt_inverse_vec = np.vectorize(dP_dt_inverse_vec_aux)

dP_dt_ICDF = np.array([dP_dt_inverse_vec(x_uniform, gamma) for gamma in gamma_uni])

dP_dt_ICDF = np.concatenate(dP_dt_ICDF)

dP_dt_ICDF_inter = interp2d(x_uniform, chi_uni_2, dP_dt_ICDF, kind='cubic')

In [ ]:
time_step = t_end/t_steps

gamma = np.array(gamma_glob)

gamma_2 = np.zeros(num_pon)

dN_dt_vec_aux = lambda gamma1: dN_dt(gamma1)

dN_dt_vec = np.vectorize(dN_dt_vec_aux)

dN_dt_inter = CubicSpline(gamma_uni, dN_dt_vec(gamma_uni))


In [ ]:
for k in tqdm(range(t_steps)):

    prob = dN_dt_inter(gamma)*time_step

    rnd_num = rng.random(num_pon)

    probability = np.greater(prob, rnd_num)

    probability = probability.astype(int)

    uni_rnd = rng.random(num_pon)

    print(dP_dt_ICDF_inter(uni_rnd, gamma))

    gamma *= (1 - dP_dt_ICDF_inter(uni_rnd, gamma))

    if (k == int(t_steps/2 - 1)):
        gamma_2 = gamma

In [ ]:
fig, axis = plt.subplots(2, 2)

fig.suptitle('Distribution Evolution of Electron Beams')

fig.tight_layout(pad=2.8)

# histograma em t=0
axis[1, 1].hist(gamma_glob / mean_ene, n_bins, density=True,
                color='red', alpha=0.5,
                histtype='bar')

# histograma em t = tend/2
axis[1, 1].hist(
    gamma_2 / mean_ene,
    n_bins, density=True,
    color='green', alpha=0.5,
    histtype='bar')

# histograma em t = tend
axis[1, 1].hist(
    gamma / mean_ene,
    n_bins, density=True,
    color='blue', alpha=0.5,
    histtype='bar')